In [1]:
import pandas as pd
import pickle

In [2]:
df_raw = pd.read_csv('../data/raw/bestsellers_with_categories.csv')
df_raw.head()

,Name,Author,User Rating,Reviews,Price,Year,Genre
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8,2016,Non Fiction
1,11/22/63: A Novel,Stephen King,4.6,2052,22,2011,Fiction
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15,2018,Non Fiction
3,1984 (Signet Classics),George Orwell,4.7,21424,6,2017,Fiction
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12,2019,Non Fiction


In [3]:
df_raw.tail()

,Name,Author,User Rating,Reviews,Price,Year,Genre
545,Wrecking Ball (Diary of a Wimpy Kid Book 14),Jeff Kinney,4.9,9413,8,2019,Fiction
546,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2016,Non Fiction
547,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2017,Non Fiction
548,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2018,Non Fiction
549,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2019,Non Fiction


Our data has been read in correctly. However, we can already see some duplicates here that we will need to fix later on. 

In [4]:
df_raw.shape

(550, 7)

In [5]:
df_raw.describe()

,User Rating,Reviews,Price,Year
count,550.000000,550.000000,550.000000,550.000000
mean,4.618364,11953.281818,13.100000,2014.000000
std,0.226980,11731.132017,10.842262,3.165156
min,3.300000,37.000000,0.000000,2009.000000
25%,4.500000,4058.000000,7.000000,2011.000000
50%,4.700000,8580.000000,11.000000,2014.000000
75%,4.800000,17253.250000,16.000000,2017.000000
max,4.900000,87841.000000,105.000000,2019.000000


In [6]:
df_raw.dtypes

Name            object
Author          object
User Rating    float64
Reviews          int64
Price            int64
Year             int64
Genre           object
dtype: object

In [7]:
df_books = df_raw.rename(columns={'User Rating': 'user_rating'})
df_books.columns = df_books.columns.str.lower()

In [8]:
df_books['genre'] = df_books['genre'].astype('category')
df_books.dtypes

name             object
author           object
user_rating     float64
reviews           int64
price             int64
year              int64
genre          category
dtype: object

In [9]:
# missing values 
df_books.isnull().sum()

name           0
author         0
user_rating    0
reviews        0
price          0
year           0
genre          0
dtype: int64

No action required on missing values, everything is filled in.


## Cleaning text columns

In [10]:
text_columns = ['name', 'author', 'genre']

for col in df_books.columns: 
    if col in text_columns: 
        n_unique_before = len(df_books[col].unique())
        n_unique_after = len(df_books[col].str.title().str.strip().unique())
        print(f'{col} before: {n_unique_before}, {col} after: {n_unique_after}')

name before: 351, name after: 350
author before: 248, author after: 248
genre before: 2, genre after: 2


We need to clean the `name` column as we have an inconsistency

In [11]:
df_books['name'] = df_books['name'].str.title().str.strip()

In [12]:
for col in df_books.columns: 
    if col in text_columns: 
        n_unique_before = len(df_books[col].unique())
        n_unique_after = len(df_books[col].str.title().str.strip().unique())
        print(f'{col} before: {n_unique_before}, {col} after: {n_unique_after}')

name before: 350, name after: 350
author before: 248, author after: 248
genre before: 2, genre after: 2


Looking at the `author` column now.

In [13]:
df_books['author'].sort_values().unique()

array(['Abraham Verghese', 'Adam Gasiewski', 'Adam Mansbach', 'Adir Levy',
       'Admiral William H. McRaven', 'Adult Coloring Book Designs',
       'Alan Moore', 'Alex Michaelides', 'Alice Schertle', 'Allie Brosh',
       'American Psychiatric Association',
       'American Psychological Association', 'Amor Towles', 'Amy Ramos',
       'Amy Shields', 'Andy Weir', 'Angie Grace', 'Angie Thomas',
       'Ann Voskamp', 'Ann Whitford Paul', 'Anthony Bourdain',
       'Anthony Doerr', 'Atul Gawande', 'Audrey Niffenegger',
       'B. J. Novak', 'Bessel van der Kolk M.D.', 'Bill Martin Jr.',
       "Bill O'Reilly", 'Bill Simmons', 'Blue Star Coloring',
       'Bob Woodward', 'Brandon Stanton', 'Brené Brown',
       'Brian Kilmeade', 'Bruce Springsteen', 'Carol S. Dweck',
       'Celeste Ng', 'Charlaine Harris', 'Charles Duhigg',
       'Charles Krauthammer', 'Cheryl Strayed', 'Chip Gaines',
       'Chip Heath', 'Chris Cleave', 'Chris Kyle', 'Chrissy Teigen',
       'Christina Baker Kline', 

As there are not many names, we can manually identify them duplicates with a quick scan. We see that George R.R Martin and J.K Rowling appear twice due to inconsistencies in spacings.

In [14]:
df_books = df_books.replace('George R.R. Martin', 'George R. R. Martin')
df_books = df_books.replace('J.K. Rowling', 'J. K. Rowling')

Checking for duplicates in the `name` column. 

In [15]:
names_list = df_books['name'].to_list()

# count each occurence of titles in names
counts_of_titles = [[title, names_list.count(title)] for title in set(names_list)]

# print duplicated titles
duplicated_titles = [title for title in counts_of_titles if title[1] > 1]
duplicated_titles

[['The Going-To-Bed Book', 2],
 ['Mockingjay (The Hunger Games)', 3],
 ['Brown Bear, Brown Bear, What Do You See?', 2],
 ['Dog Man: Brawl Of The Wild: From The Creator Of Captain Underpants (Dog Man #6)',
  2],
 ['Wonder', 5],
 ["Giraffes Can'T Dance", 5],
 ['Wheat Belly: Lose The Wheat, Lose The Weight, And Find Your Path Back To Health',
  2],
 ['School Zone - Big Preschool Workbook - Ages 4 And Up, Colors, Shapes, Numbers 1-10, Alphabet, Pre-Writing, Pre-Reading…',
  2],
 ['How To Win Friends & Influence People', 5],
 ['The Book Thief', 2],
 ['The Immortal Life Of Henrietta Lacks', 3],
 ['Laugh-Out-Loud Jokes For Kids', 5],
 ["Proof Of Heaven: A Neurosurgeon'S Journey Into The Afterlife", 2],
 ['The Help', 4],
 ['Becoming', 2],
 ['The Pout-Pout Fish', 2],
 ['The Wonky Donkey', 2],
 ['The Last Olympian (Percy Jackson And The Olympians, Book 5)', 2],
 ['The Girl With The Dragon Tattoo (Millennium Series)', 2],
 ['Between The World And Me', 2],
 ['Steve Jobs', 2],
 ['Ready Player One: 

Let's check a couple of these below.

In [16]:
df_books[df_books['name'] == 'Gone Girl']

,name,author,user_rating,reviews,price,year,genre
135,Gone Girl,Gillian Flynn,4.0,57271,10,2012,Fiction
136,Gone Girl,Gillian Flynn,4.0,57271,10,2013,Fiction
137,Gone Girl,Gillian Flynn,4.0,57271,9,2014,Fiction


In [17]:
df_books[df_books['name'] == 'Steve Jobs']

,name,author,user_rating,reviews,price,year,genre
301,Steve Jobs,Walter Isaacson,4.6,7827,20,2011,Non Fiction
302,Steve Jobs,Walter Isaacson,4.6,7827,20,2012,Non Fiction


We see that we have duplicate titles occurring on the `price` and `year` columns. We will set `price` to equal the average price of the book across all years, and we will select the most recent `year`.

In [18]:
df_books['price'] = df_books.groupby('name')['price'].transform('mean')
df_books['year'] = df_books.groupby('name')['year'].transform('max')

In [19]:
df_books = df_books.drop_duplicates(keep='last')

After dropping these duplicates, we'll run our check again. 

In [20]:
# Run our check again
names_list = df_books['name'].to_list()

counts_of_titles = [[title, names_list.count(title)] for title in set(names_list)]
duplicated_titles = [title for title in counts_of_titles if title[1] > 1]
print(duplicated_titles)

[['The 7 Habits Of Highly Effective People: Powerful Lessons In Personal Change', 2], ['The 5 Love Languages: The Secret To Love That Lasts', 2]]


In [21]:
df_books[df_books['name'] == 'The 5 Love Languages: The Secret To Love That Lasts']

,name,author,user_rating,reviews,price,year,genre
324,The 5 Love Languages: The Secret To Love That ...,Gary Chapman,4.7,3477,18.0,2019,Non Fiction
329,The 5 Love Languages: The Secret To Love That ...,Gary Chapman,4.8,25554,18.0,2019,Non Fiction


In [22]:
df_books[df_books['name'] == 'The 7 Habits Of Highly Effective People: Powerful Lessons In Personal Change']

,name,author,user_rating,reviews,price,year,genre
334,The 7 Habits Of Highly Effective People: Power...,Stephen R. Covey,4.6,9325,20.571429,2017,Non Fiction
337,The 7 Habits Of Highly Effective People: Power...,Stephen R. Covey,4.7,4725,20.571429,2017,Non Fiction


It's possible that these are genuine duplicated entries, going back and looking at them we can see that it appears to be two versions of the book released with two different prices, leading to different numbers of reviews. We can't combine them here, nor can we say that one is more important than the other. For the purposes of this exercise we will simply drop these rows. 


In [23]:
df_books.drop(
    df_books[df_books['name'] == 'The 5 Love Languages: The Secret To Love That Lasts'].index,
    inplace=True
)

df_books.drop(
    df_books[df_books['name'] == 'The 7 Habits Of Highly Effective People: Powerful Lessons In Personal Change'].index,
    inplace=True
)

In [24]:
df_books.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 348 entries, 0 to 549
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   name         348 non-null    object  
 1   author       348 non-null    object  
 2   user_rating  348 non-null    float64 
 3   reviews      348 non-null    int64   
 4   price        348 non-null    float64 
 5   year         348 non-null    int64   
 6   genre        348 non-null    category
dtypes: category(1), float64(2), int64(2), object(2)
memory usage: 19.5+ KB


In [25]:
df_books.describe()

,user_rating,reviews,price,year
count,348.000000,348.000000,348.000000,348.000000
mean,4.608046,9773.945402,12.974282,2014.097701
std,0.227361,10880.091401,10.025634,3.379323
min,3.300000,37.000000,0.000000,2009.000000
25%,4.500000,3406.250000,8.000000,2011.000000
50%,4.600000,6328.000000,11.000000,2014.000000
75%,4.800000,11430.750000,16.000000,2017.000000
max,4.900000,87841.000000,105.000000,2019.000000


In [26]:
print(f"Number of books: {df_books.shape[0]}, number of authors: {len(df_books['author'].unique())}")

Number of books: 348, number of authors: 245


Our final dataset now has 348 books written by 245 different authors. We can now move on to visualizing this data. 

In [27]:
df_books.to_pickle('../data/processed/bestsellers_cleaned')